# Embeddings, Vector Databases, and Search

Converting text into embedding vectors is the first step to any text processing pipeline. As the amount of text gets larger, there is often a need to save these embedding vectors into a dedicated vector index or library, so that developers won't have to recompute the embeddings and the retrieval process is faster. We can then search for documents based on our intended query and pass these relevant documents into a language model (LM) as additional context. We also refer to this context as supplying the LM with "state" or "memory". The LM then generates a response based on the additional context it receives!


In this notebook, we will implement the full workflow of text vectorization, vector search, and question answering workflow. While we use [FAISS](https://faiss.ai/) (vector library) and [ChromaDB](https://docs.trychroma.com/) (vector database), and a Hugging Face model, know that you can easily swap these tools out for your preferred tools or models!
<img src="https://files.training.databricks.com/images/llm/updated_vector_search.png" width=1000 target="_blank" >

Learning Objectives
1. Implement the workflow of reading text, converting text to embeddings, saving them to FAISS and ChromaDB
2. Query for similar documents using FAISS and ChromaDB
3. Apply a Hugging Face language model for question answering!

In [ ]:
!pip install faiss-cpu==1.7.4 chromadb==0.3.21

## Step 1: Reading data

In this section, we are going to use the data on <a href="https://newscatcherapi.com/" target="_blank">news topics collected by the NewsCatcher team</a>, who collect and index news articles and release them to the open-source community. The dataset can be downloaded from <a href="https://www.kaggle.com/kotartemiy/topic-labeled-news-dataset" target="_blank">Kaggle</a>.


In [21]:
import pandas as pd
df = pd.read_csv("sample_data/labelled_newscatcher_dataset.csv",sep=";")
df["id"] = df.index
display(df)

,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,3
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,4
...,...,...,...,...,...,...,...
108769,NATION,https://www.vanguardngr.com/2020/08/pdp-govern...,vanguardngr.com,2020-08-08 02:40:00,PDP governors’ forum urges security agencies t...,en,108769
108770,BUSINESS,https://www.patentlyapple.com/patently-apple/2...,patentlyapple.com,2020-08-08 01:27:12,"In Q2-20, Apple Dominated the Premium Smartpho...",en,108770
108771,HEALTH,https://www.belfastlive.co.uk/news/health/coro...,belfastlive.co.uk,2020-08-12 17:01:00,Coronavirus Northern Ireland: Full breakdown s...,en,108771
108772,ENTERTAINMENT,https://www.thenews.com.pk/latest/696364-paul-...,thenews.com.pk,2020-08-05 04:59:00,Paul McCartney details post-Beatles distress a...,en,108772


## Vector Library: FAISS

Vector libraries are often sufficient for small, static data. Since it's not a full-fledged database solution, it doesn't have the CRUD (Create, Read, Update, Delete) support. Once the index has been built, if there are more vectors that need to be added/removed/edited, the index has to be rebuilt from scratch.

That said, vector libraries are easy, lightweight, and fast to use. Examples of vector libraries are [FAISS](https://faiss.ai/), [ScaNN](https://github.com/google-research/google-research/tree/master/scann), [ANNOY](https://github.com/spotify/annoy), and [HNSM](https://arxiv.org/abs/1603.09320).

FAISS has several ways for similarity search: L2 (Euclidean distance), cosine similarity. You can read more about their implementation on their [GitHub](https://github.com/facebookresearch/faiss/wiki/Getting-started#searching) page or [blog post](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/). They also published their own [best practice guide here](https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index).

If you'd like to read up more on the comparisons between vector libraries and databases, [here is a good blog post](https://weaviate.io/blog/vector-library-vs-vector-database#feature-comparison---library-versus-database).


The overall workflow of FAISS is captured in the diagram below.

<img src="https://miro.medium.com/v2/resize:fit:1400/0*ouf0eyQskPeGWIGm" width=700>

Source: [How to use FAISS to build your first similarity search by Asna Shafiq](https://medium.com/loopio-tech/how-to-use-faiss-to-build-your-first-similarity-search-bf0f708aa772).


In [5]:
from sentence_transformers import InputExample

sample_data = df.head(1000)

def example_create_fn(doc: pd.Series) -> InputExample:
  return InputExample(texts=[doc])

faiss_training_examples = sample_data.apply(lambda x: example_create_fn(x['title']),axis=1).to_list()

In [7]:
faiss_training_examples[0]

### Step 2: Vectorize text into embedding vectors

We will be using `Sentence-Transformers` [library](https://www.sbert.net/) to load a language model to vectorize our text into embeddings. The library hosts some of the most popular transformers on [Hugging Face Model Hub](https://huggingface.co/sentence-transformers).

Here, we are using the `model = SentenceTransformer("all-MiniLM-L6-v2")` to generate embeddings.


In [22]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-V2", cache_folder="/sample_data")
faiss_title_embeddings = embedding_model.encode(sample_data.title.values.tolist())


In [23]:
print(len(faiss_title_embeddings))
print(len(faiss_title_embeddings[0]))


1000
384


### Step 3: Saving embedding vectors to FAISS index

Below, we create the FAISS index object based on our embedding vectors, normalize vectors, and add these vectors to the FAISS index.


In [24]:
import numpy as np
import faiss

pdf_to_index = sample_data.set_index(["id"], drop=False)
id_index = np.array(pdf_to_index.id.values).flatten().astype("int")

content_encoded_normalized = faiss_title_embeddings.copy()
faiss.normalize_L2(content_encoded_normalized)

# Index1DMap translates search results to IDs: https://faiss.ai/cpp_api/file/IndexIDMap_8h.html#_CPPv4I0EN5faiss18IndexIDMapTemplateE
# The IndexFlatIP below builds index
index_content = faiss.IndexIDMap(faiss.IndexFlatIP(len(faiss_title_embeddings[0])))
index_content.add_with_ids(content_encoded_normalized, id_index)

In [29]:
def search_content(query, pdf_to_index, k=3):
    query_vector = embedding_model.encode([query])
    faiss.normalize_L2(query_vector)

    # We set k to limit the number of vectors we want to return
    top_k = index_content.search(query_vector, k)
    ids = top_k[1][0].tolist()
    similarities = top_k[0][0].tolist()
    results = pdf_to_index.loc[ids]
    results["similarities"] = similarities
    return results


In [31]:
display(search_content("animal",pdf_to_index))

,topic,link,domain,published_date,title,lang,id,similarities
id,,,,,,,,
176,TECHNOLOGY,https://www.pushsquare.com/news/2020/08/random...,pushsquare.com,2020-08-03 16:30:00,Random: You Can Pick Up and Pet Cats in Assass...,en,176,0.391902
975,HEALTH,https://www.news-medical.net/news/20200813/Res...,news-medical.net,2020-08-13 05:18:00,Researchers explore social behavior of animals...,en,975,0.376784
99,TECHNOLOGY,https://www.gematsu.com/2020/08/ghostwire-toky...,gematsu.com,2020-08-07 16:43:13,Ghostwire: Tokyo confirms dog petting,en,99,0.344058


###FAISS through LangChain

In [52]:

from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings

#FAISS.from_documents - Return VectorStore initialized from documents and embeddings.
vector_store = FAISS.from_texts(sample_data['title'],SentenceTransformerEmbeddings())

question = "animal"

#similar_docs = vector_store.similarity_search(question, k=2)

similar_docs_with_score = vector_store.similarity_search_with_score(question, k=3)

print(similar_docs_with_score)

[(Document(page_content='Researchers explore social behavior of animals toward emerging infectious diseases', metadata={}), 1.247191), (Document(page_content='Just Let This Lizard Be a Dinosaur', metadata={}), 1.3853452), (Document(page_content="Random: You Can Pick Up and Pet Cats in Assassin's Creed Valhalla", metadata={}), 1.4018977)]


In [53]:
print(similar_docs_with_score[0][0].page_content)
print(similar_docs_with_score[1][0].page_content)
print(similar_docs_with_score[2][0].page_content)

Researchers explore social behavior of animals toward emerging infectious diseases
Just Let This Lizard Be a Dinosaur
Random: You Can Pick Up and Pet Cats in Assassin's Creed Valhalla


Up until now, we haven't done the last step of conducting Q/A with a language model yet. We are going to demonstrate this with Chroma, a vector database.

## Vector Database: Chroma

Chroma is an open-source embedding database. The company just raised its [seed funding in April 2023](https://www.trychroma.com/blog/seed) and is quickly becoming popular to support LLM-based applications.


In [59]:
import chromadb
from chromadb.config import Settings

chroma_client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",persist_directory="/sample_data"))

In [ ]:
collection_name = "my_news"

# If you have created the collection before, you need to delete the collection first
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
    chroma_client.delete_collection(name=collection_name)

print(f"Creating collection: '{collection_name}'")
collection = chroma_client.create_collection(name=collection_name)

In [61]:
collection.add(
    documents=sample_data["title"][:100].tolist(),
    metadatas=[{"topic": topic} for topic in sample_data["topic"][:100].tolist()],
    ids=[f"id{x}" for x in range(100)],
)

In [66]:
import json

results = collection.query(query_texts=["space"], n_results=10)

print(json.dumps(results, indent=4))

{
    "ids": [
        [
            "id72",
            "id7",
            "id30",
            "id26",
            "id23",
            "id76",
            "id69",
            "id40",
            "id47",
            "id75"
        ]
    ],
    "embeddings": null,
    "documents": [
        [
            "Beck teams up with NASA and AI for 'Hyperspace' visual album experience",
            "Orbital space tourism set for rebirth in 2021",
            "NASA drops \"insensitive\" nicknames for cosmic objects",
            "\u2018It came alive:\u2019 NASA astronauts describe experiencing splashdown in SpaceX Dragon",
            "Hubble Uses Moon As \u201cMirror\u201d to Study Earth\u2019s Atmosphere \u2013 Proxy in Search of Potentially Habitable Planets Around Other Stars",
            "Australia's small yet crucial part in the mission to find life on Mars",
            "NASA Astronauts in SpaceX Capsule Splashdown in Gulf Of Mexico",
            "SpaceX's Starship spacecraft saw 150 mete

In [64]:
collection.query(query_texts=["space"], where={"topic": "SCIENCE"}, n_results=10)

{'ids': [['id16',
   'id1',
   'id26',
   'id36',
   'id19',
   'id18',
   'id27',
   'id20',
   'id2',
   'id38']],
 'embeddings': None,
 'documents': [['Tradeoff between the eyes and nose helps flies find their niche',
   'An irresistible scent makes locusts swarm, study finds',
   '‘It came alive:’ NASA astronauts describe experiencing splashdown in SpaceX Dragon',
   'Cause Of Massive Fish Kill In Shinnecock Canal Not Clear - 27 East',
   'Nasa SpaceX crew return: Dragon capsule splashes down',
   'In rare find, fossil shows how Cretaceous-era ‘hell ant’ ate its prey with weird jaws',
   'When did humans find out how to use fire?',
   'Take a look at Mar’s eerie nightglow',
   'Artificial intelligence warning: AI will know us better than we know ourselves',
   "'Zombie cicadas' under the influence of a mind controlling fungus have returned to West Virginia"]],
 'metadatas': [[{'topic': 'SCIENCE'},
   {'topic': 'SCIENCE'},
   {'topic': 'SCIENCE'},
   {'topic': 'SCIENCE'},
   {'topic

In [ ]:
pip install accelerate

In [81]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id="gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="/sample_data")
lm_model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir="/sample_data")
pipeline_translation  = pipeline(
    task="text-generation",
    model=lm_model,
    tokenizer=tokenizer,
        max_new_tokens=512,


)

In [74]:
question = "What's the latest news on space development?"
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"

In [82]:
lm_response = pipeline_translation(prompt_template)
print(lm_response[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Relevant context: #Beck teams up with NASA and AI for 'Hyperspace' visual album experience #Orbital space tourism set for rebirth in 2021 #NASA drops "insensitive" nicknames for cosmic objects #‘It came alive:’ NASA astronauts describe experiencing splashdown in SpaceX Dragon #Hubble Uses Moon As “Mirror” to Study Earth’s Atmosphere – Proxy in Search of Potentially Habitable Planets Around Other Stars #Australia's small yet crucial part in the mission to find life on Mars #NASA Astronauts in SpaceX Capsule Splashdown in Gulf Of Mexico #SpaceX's Starship spacecraft saw 150 meters high #NASA’s InSight lander shows what’s beneath Mars’ surface #Alien base on Mercury: ET hunters claim to find huge UFO

 The user's question: What's the latest news on space development? This section has received a lot of feedback.

Posting Rules You may not post new threads You may not post replies You may not post attachments You may not edit your posts On BB code is On Smilies are On [IMG] code is HTML cod